In [8]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point

In [9]:
## calculate population number in 2021
population_data = pd.read_csv('../data/curated/SA2.csv')
population_data = population_data[population_data['ASGS_2021: Region'] == '2: Victoria']
region_population_2021 = population_data[population_data['TIME_PERIOD: Time Period'] == 2021]['OBS_VALUE'].sum()
print(region_population_2021)



25688079


In [10]:
# Reads in shape file downloaded from: https://discover.data.vic.gov.au/dataset/postcode-boundaries-polygon-vicmap-admin
sf = gpd.read_file("../data/raw/rental_zones/POSTCODE_POLYGON.shp")

In [11]:
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")



In [12]:
f = open("../data/raw/scraped_rental.json")
rental_data = json.load(f)
rental_df = pd.DataFrame.from_dict(rental_data, orient = 'index')

In [13]:
# Makes all the coordinates scraped into (lattitude ,longitude)
rental_df['coordinates'] = rental_df['coordinates'].apply(lambda x: list(reversed(x)))
rental_df['coordinates'] = rental_df['coordinates'].apply(Point)

In [14]:
#Creates geodataframe from the shapefile with postcodes
postcode_sf = gpd.GeoDataFrame.from_file("../data/raw/rental_zones/POSTCODE_POLYGON.shp")
postcode_sf['geometry'] = postcode_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [15]:
# Creates geodataframe with points of rental properties
rental_points = gpd.GeoDataFrame(rental_df,geometry='coordinates',crs = postcode_sf.crs)

In [16]:
# Merges two geodataframes and checkes if properties are in postcode area
pointInPolys = gpd.tools.sjoin(rental_points, postcode_sf, how='left')

In [27]:
# calculate population of postcode by ratio of rent info in every post code suburb
rent_by_postcode = pointInPolys.groupby('POSTCODE').count()
all = rent_by_postcode['name'].sum()
ratio = rent_by_postcode['name'] / all
rent_by_postcode['POSTCODE_POPULATION'] = ratio * region_population_2021
rent_by_postcode['POSTCODE_POPULATION'].apply(lambda x: int(x))
print(rent_by_postcode)

          name  cost_text  coordinates  rooms  desc  index_right  PFI  PFI_CR  \
POSTCODE                                                                        
3000        22         22           22     22    22           22   22      22   
3002         6          6            6      6     6            6    6       6   
3003         2          2            2      2     2            2    2       2   
3004        16         16           16     16    16           16   16      16   
3006        16         16           16     16    16           16   16      16   
...        ...        ...          ...    ...   ...          ...  ...     ...   
3939         2          2            2      2     2            2    2       2   
3941         1          1            1      1     1            1    1       1   
3942         1          1            1      1     1            1    1       1   
3943         1          1            1      1     1            1    1       1   
3944         4          4   